# 1. 移动坐标点

## 1.1 走出第一步

有了大小和方向，接下来我们就可以走出我们的第一步了。来看权重的迭代公式：

$$
\boldsymbol{w}_{(t+1)}=\boldsymbol{w}_{(t)}-\eta \frac{\partial L(w)}{\partial \boldsymbol{w}}
$$

现在我们的偏导数部分已经计算出来了，就是我们用backward求解出的结果。而 $ \eta $ 学习率，或者步长，是我们在迭代开始之前就认为设置好的，一般是0.01-0.5之间的某个小数，因此我们已经可以无障碍地使用代码实现权重的迭代了：


In [1]:
# 在这里，我们的数据是生成的随机数，为了让大家看出效果，所以设置步长为10，正常不会使用这么长的步长
# 步长、学习率的英文是learning rate，所以常常简写为lr

import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(420)
X = torch.rand((500, 20), dtype= torch.float32)*100
y = torch.randint(low= 0, high= 3, size=(500,), dtype= torch.float32)
in_features = X.shape[1]
out_features = len(y.unique())

class Model(nn.Module):
  def __init__(self, in_features= 10, out_features=2):
    super().__init__()
    self.linear_1 = nn.Linear(in_features, 13, bias= False)
    self.linear_2 = nn.Linear(13, 8, bias = False)
    self.out = nn.Linear(8, out_features, bias = False)

  def forward(self, X):
    z1 = self.linear_1(X)
    sigma_1 = F.relu(z1)
    z2 = self.linear_2(sigma_1)
    sigma_2 = F.sigmoid(z2)
    zhat = self.out(sigma_2)
    return zhat

torch.manual_seed(420)
net = Model(in_features, out_features)
zhat = net.forward(X)
criterion = nn.CrossEntropyLoss()
loss = criterion(zhat, y.long())
loss.backward()
w = net.linear_1.weight.data

In [2]:
lr = 10
dw = net.linear_1.weight.grad
w -= lr*dw # type: ignore
w

tensor([[ 1.3728e-01, -1.3441e-01,  2.1369e-01, -1.7753e-01, -6.7905e-02,
         -1.5396e-01,  1.7325e-01,  8.4504e-02, -1.1130e-01, -1.7284e-01,
         -1.2928e-01, -4.2774e-02, -1.1391e-01,  1.6372e-01, -9.3821e-02,
         -1.4613e-01, -6.8671e-02, -2.1835e-01, -1.0836e-01, -1.2178e-01],
        [-1.2292e-01,  1.0665e-01, -1.6468e-01, -2.0308e-01, -3.5176e-02,
         -1.9426e-01, -2.2805e-01, -1.5618e-01, -3.2817e-01, -1.4099e-01,
          3.6728e-02, -1.1321e-01, -8.2137e-02, -2.5312e-01, -3.1246e-01,
         -1.8933e-01, -2.4277e-01, -4.0041e-02, -2.6232e-01,  1.2886e-02],
        [-7.3430e-02, -3.6251e-02,  1.2690e-01, -3.6448e-02,  4.1469e-02,
          1.4727e-01,  1.3623e-01, -1.7868e-01,  1.5008e-01,  1.9492e-01,
          1.7009e-01, -2.6587e-01, -1.0714e-01, -1.7380e-01,  1.4086e-01,
          1.4875e-01, -1.1610e-01,  2.7153e-01, -2.1605e-01, -5.7652e-03],
        [ 2.8962e-01,  5.5489e-02,  8.9793e-02,  2.9763e-01, -8.9448e-02,
          2.0657e-01, -1.8569e-02, 

## 1.2 第一步到第二步：动量法Momentum

普通梯度下降就是在重复正向传播，计算梯度，更新权重的过程，但这个过程往往非常漫长。如大家所见，步长设置为0.001时，我们看不到 w 任何变化，只有当步长设置得非常巨大，我们才能够看到一些改变，但在之前的过程我们说过，巨大的步长可能会让我们跳过真正的最小值，所以我们无法将步长设置得非常大，无论如何，梯度下降都是一个非常慢的过程。在这个基础上，我们提出了加速迭代的数个方法，其中一个很关键的方法，就是使用动量Momentum。

之前我们说过，在梯度下降过程中，起始点是一个“盲人”，它看不到也听不到全局，所以我们每移动一次都要重新计算方向与距离，并且每次只能走一小步。但不只限于次，起始点不仅看不到前面的路，也无法从过去走过的路中学习。

想象一下，我们被蒙上眼睛，由另一个人喊口号来给我们方向让我们移动，假设喊口号的人一直喊：“向前，向前，向前”。因为我们看不见，在最初的三四次，我们可能只会向前走一小步，但如果他一直让我们向前，我们就会失去耐心，转而向前走一大步，因为我们可以预测：前面很长一段路大概都是需要向前的。对梯度下降来说，也是同样的道理——如果在很长一段时间内，起始点一直向着相似的方向移动，那按照步长一小步一小步地磨着向前走是没有意义的，既浪费计算资源又浪费时间，此时就应该大胆地按照这个方向走一大步。相对的，如果我们很长时间都走向同一个方向，突然让我们转向，那我们转向的第一步就应该非常谨慎，走一小步。

不难发现，真正高效的方法是：在历史方向与现有方向相同的情况下，迈出大步子，在历史方向与现有方向相反的情况下，迈出小步子。那要怎么才能让起始点了解过去的方向呢？**我们让上一步的梯度向量（的反方向）与现在这一点的梯度向量（的反方向）以加权的方式求和，求解出受到上一步大小和方向影响的真实下降方向，再让坐标点向真实下降方向移动**。在坐标轴上可以表示为：

![Alt text](image-20.png)

其中，对于上一步的梯度向量加上的权重被称为动量参数（也叫做衰减力度，通常使用 $ \gamma $ 进行表示），对这一点的梯度向量加上权重就是步长（依然为 $ \eta $）,真实移动的向量为 $ v $，被称为动量。将上述过程使用公式表示，则有：

$$
\begin{aligned}
v_{(t)} & =\gamma v_{(t-1)}-\eta \frac{L}{\partial \boldsymbol{w}} \\
\boldsymbol{w}_{(t+1)} & =\boldsymbol{w}_{(t)}+v_{(t)}
\end{aligned}
$$

在第一步中，没有历史梯度方向，因此第一步的真实方向就是起始点梯度的反方向， $ v_{0} = 0$ 。其中$ v_{(t-1)}$ 代表了之前所有步骤所累计的动量和（其实也就代表着上一步的真实方向）。在这种情况下，梯度下降的方向有了惯性，受到历史累积动量的影响，当新坐标点的梯度反方向与历史累积动量的方向一致时，历史累积动量会加大实际方向的步子，相反，当新坐标点的梯度方向与历史累积动量的方向不一致时，历史累积动量会减小实际方向的步子。

我们可以很容易地在PyTorch中实现动量法：



In [3]:
# v(t) = gamma * v(t-1) - lr * dw
# w(t+1) = w(t) + v(t)

lr = 0.1
gamma = 0.9
dw = net.linear_1.weight.grad.data
w = net.linear_1.weight.data

# t = 1,走出第一步，进行首次迭代的时候，需要一个v0
v = torch.zeros(dw.shape[0], dw.shape[1])


In [4]:
v = gamma*v - lr*dw
w = w + v
w

tensor([[ 0.1373, -0.1344,  0.2137, -0.1775, -0.0679, -0.1540,  0.1733,  0.0845,
         -0.1113, -0.1728, -0.1293, -0.0428, -0.1139,  0.1637, -0.0938, -0.1461,
         -0.0687, -0.2184, -0.1084, -0.1218],
        [-0.1246,  0.1059, -0.1666, -0.2038, -0.0364, -0.1953, -0.2293, -0.1573,
         -0.3303, -0.1426,  0.0353, -0.1147, -0.0843, -0.2537, -0.3138, -0.1909,
         -0.2452, -0.0418, -0.2630,  0.0121],
        [-0.0722, -0.0370,  0.1267, -0.0371,  0.0412,  0.1473,  0.1354, -0.1791,
          0.1508,  0.1950,  0.1705, -0.2665, -0.1067, -0.1734,  0.1413,  0.1480,
         -0.1151,  0.2725, -0.2161, -0.0056],
        [ 0.2907,  0.0564,  0.0904,  0.2990, -0.0885,  0.2081, -0.0177,  0.3475,
         -0.1296,  0.1525,  0.2200, -0.0655, -0.1232,  0.2037, -0.0881,  0.2521,
          0.0986, -0.1092,  0.2526, -0.0839],
        [ 0.1828,  0.2618,  0.0668,  0.2342,  0.0424, -0.0886, -0.0956,  0.3314,
          0.1528,  0.2811, -0.2191,  0.1931,  0.0139, -0.1140, -0.0872,  0.2661,
      

## 1.3 torch.optim实现带动量的梯度下降

在PyTorch库的架构中，拥有专门实现优化算法的模块torch.optim。我们在之前的课程中所说的迭代流程，都可以通过torch.optim模块来简单地实现。

接下来，我们就基于之前定义的类Model来实现梯度下降的一轮迭代：



In [ ]:
# 导入库
# 确定数据，超参数的确定(lr, gamma)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F

# 确定数据
torch.manual_seed(420)
X = torch.rand(500, 20, dtype= torch.float32)*100
y = torch.randint(low= 0, high= 3, size= (500,), dtype= torch.float32)

In [ ]:
# 定义超参数
lr = 0.1
gamma = 0.9

In [ ]:
# 定义神经网络的架构

class Model(nn.Module):
  def __init__(self, in_features= 10, out_features=2):
    super().__init__()
    self.linear_1 = nn.Linear(in_features, 13, bias= True)
    self.linear_2 = nn.Linear(13, 8, bias = True)
    self.out = nn.Linear(8, out_features, bias = True)

  def forward(self, X):
    z1 = self.linear_1(X)
    sigma_1 = F.relu(z1)
    z2 = self.linear_2(sigma_1)
    sigma_2 = F.sigmoid(z2)
    zhat = self.out(sigma_2)
    return zhat


In [ ]:
input = X.shape[1]
output = len(y.unique())

In [ ]:
# 实例化神经网络
torch.manual_seed(420)
net = Model(input, output)

In [ ]:
# 定义损失函数
criterion = nn.CrossEntropyLoss()

In [ ]:
# 定义优化算法
opt = optim.SGD(net.parameters()
         ,lr = lr
         ,momentum = gamma)

In [ ]:
# 向前传播
# 本轮向前传播的损失函数值
# 反向传播 - 得到梯度
# 更新权重(和动量)
# 清空梯度- 清除原来计算出来的，基于上一个点的坐标计算的梯度

zhat = net.forward(X)
loss = criterion(zhat, y.long())
loss.backward()
opt.step()    # 步子，走一步，更新权重W，更新动量V
opt.zero_grad()

print(loss)
print(net.linear_1.weight.data[0][:10])